# Lab 1: Set Covering
---
## Path Search

### Description of the problem
Given the number of sets ``NUM_SETS`` and the number of elements inside each set ``PROBLEM_SIZE``, determine, if possible, the collection of sets through which all the elements are available.

A state is made up of the sets of items that I take and the sets that I don't take. 
* State=({1,3,5}, {0,2,4,6,7}) -> I'm taking the second array of items from ``SETS``, the fourth and the sixth.
* The quality of a solution (the chosen state) is given by the smallest number of taken sets to get all the elements

### Objectives

**1 - Implementation of Search Algorithms**
The goal of the project is to find an algorithm that can efficiently solve the Set Covering Problem. 

**2 - Build a Heuristic Function H for A star**
The focus of the laboratory is on the A* search algorithm and therefore on finding a Heuristic Function H.
To be sure to find the optimal solution, the heuristic function must be:
- Admissible: It never overestimates the cost of reaching the goal, so that $H(n) \leq H^*(n)$
- Consistent: It satisfies the triangle inequality, so that $H(n) \leq c(n, a, n') + H(n')$

# Code
---

### Imported Libraries

In [1]:
from random import random, choice, randint
from functools import reduce
from math import ceil
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np
from tqdm.auto import tqdm
from copy import copy

### Problem instance
We implement the sets as an array of arrays where one element has a 20% chance of being true. A set indicates which item is inside the set and which is not, if an element of the set has a value of true it means that it is present otherwise not

In [16]:
PROBLEM_SIZE = 50
NUM_SETS = 100
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

### Functions for algorithms

- ``goal_check`` is the function that checks whether the state given in input is a solution, that is if all the elements are present within the states taken. To do that, we need to do an or between all the elements of all the sets of the state to see if a certain element is present among all the sets or not (true is present, false is not). With ``all``, we check if all the elements are present, that is, if the ``reduce`` gives me an array of True which means that all the elements are present among the sets taken 
- ``distance`` is the function that calculate the distance from the input state to the final goal (for a greedy search), in short, it returns how many elements that need to be taken are missing
- ``covered`` is the function that returns what are the covered elements in the taken sets

In [4]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


def covered(state):
    return reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))

In [5]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

# Breadth-first search algorithm
Implementation using breadth-first search algorithm using a Simple Queue which would be a FIFO
If we used a LifoQueue it would become a depth-first search

In [ ]:
frontier = SimpleQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(current_state.taken ^ {action},
                              current_state.not_taken ^ {action})  # {1,2,3} ^ {2} = {1,3}  {1,3} ^ {2} = {1,2,3}
            frontier.put(new_state)
        current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

# Dijkstra search algorithm
The cost used for the PriorQueue is the number of elements in a set i.e., those taken

In [ ]:
frontier = PriorityQueue()
frontier.put((0, (State(set(), set(range(NUM_SETS))))))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(current_state.taken ^ {action},
                              current_state.not_taken ^ {action})  # {1,2,3} ^ {2} = {1,3}  {1,3} ^ {2} = {1,2,3}
            frontier.put((len(new_state.taken), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

# Greedy search algorithm

In [ ]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(current_state.taken ^ {action},
                              current_state.not_taken ^ {action})  # {1,2,3} ^ {2} = {1,3}  {1,3} ^ {2} = {1,2,3}
            frontier.put((distance(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

# A* search algorithm

The ``distance`` function is not usable as heuristic for A* because it is pessimistic. If it returns that the distance from the goal is three, I'm sure that I can cover them with three sets, but it is the most pessimistic solution and the heuristic for A* has to be optimistic. These are the functions that I tried ordered from the least performing to the most performing:

- ``h1``: if the largest set (the one with most elements) covers two elements and if ``missing_size`` is three (the number of elements there are missing), then I will need at least two sets (3/2). 
- ``h2``: the same as h1 but for the largest set I take into account only the items that have not yet been covered. ``largest_remaining_set_size`` it will therefore be the maximum number of uncovered elements present in a set.
- ``h3``: instead of considering only the set with the maximum size, we use the size of all uncovered sets and sort it taking into account, for the dimension of them, only the items that have not yet been covered. So for example, if I am missing four elements and in the list I have five sets with ordered dimensions (``candidates`` = [2, 1, 1, 1, 1]), it means that I will need at least three sets (2+1+1 = 4)

In [20]:
def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_remaining_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_remaining_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def A_star(state):
    return len(state.taken) + h1(state)

In [ ]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((A_star(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((A_star(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")